In [1]:
# Third-Party Tools: Expanding Horizons with Tavily and Google ADK
# Welcome 👋🏻 In this notebook, you'll learn how to expand your ADK agent's capabilities by integrating 
# a third-party tool—Tavily—for live web search. You'll see how to wrap LangChain tools for ADK, 
# configure your agent, and run real queries for up-to-date information in WanderWise.

In [2]:
# Install Google ADK for Python
# This foundational package provides all the necessary components for building and running your agents.
# The --quiet flag suppresses verbose output during installation.
%pip install google-adk --quiet


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Install LangChain Community and Tavily Python libraries
# langchain_community provides LangChain integrations, including tools.
# tavily-python is the official Python client for the Tavily Search API.
%pip install langchain_community tavily-python --quiet


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Verify ADK Installation (Optional but Recommended)
%pip show google-adk
%pip show langchain_community 
%pip show tavily-python

Name: google-adk
Version: 1.0.0
Summary: Agent Development Kit
Home-page: https://google.github.io/adk-docs/
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: 
Location: /home/codespace/.python/current/lib/python3.12/site-packages
Requires: authlib, click, fastapi, google-api-python-client, google-cloud-aiplatform, google-cloud-secret-manager, google-cloud-speech, google-cloud-storage, google-genai, graphviz, mcp, opentelemetry-api, opentelemetry-exporter-gcp-trace, opentelemetry-sdk, pydantic, python-dotenv, PyYAML, sqlalchemy, tzlocal, uvicorn
Required-by: 
Note: you may need to restart the kernel to use updated packages.
Name: langchain-community
Version: 0.3.24
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /home/codespace/.python/current/lib/python3.12/site-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, reque

In [ ]:
# Configure environment
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# Load API key from .env file
# Create a .env file with: GOOGLE_GENAI_API_KEY=your-api-key-here
api_key = os.environ.get("GOOGLE_GENAI_API_KEY")

if not api_key:
    raise ValueError(
        "GOOGLE_GENAI_API_KEY not found.\n"
        "Create a .env file with: GOOGLE_GENAI_API_KEY=your-api-key-here\n"
        "Get your API key from: https://aistudio.google.com/apikey"
    )

os.environ["GEMINI_API_KEY"] = api_key

print(f"✓ API key configured (length: {len(api_key)} characters)")
print(f"✓ GOOGLE_GENAI_USE_VERTEXAI = {os.environ.get('GOOGLE_GENAI_USE_VERTEXAI')}")

# Define the specific Gemini model we'll use.
# 'gemini-2.0-flash' is a fast and efficient model, suitable for quick responses.
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

In [6]:
# Import Necessary Modules

# Agent: The fundamental class for creating an AI agent within ADK.
from google.adk.agents import Agent

# Runner: The orchestrator that manages the flow of messages and agent execution.
from google.adk.runners import Runner

# InMemorySessionService: A simple, in-memory service for managing conversation sessions.
# Sessions are vital for maintaining context in multi-turn dialogues.
from google.adk.sessions import InMemorySessionService

# LangchainTool: A utility from ADK to wrap LangChain tools, making them compatible with ADK agents.
from google.adk.tools.langchain_tool import LangchainTool

# types from google.genai: Provides data structures (like Content and Part) to represent
# messages exchanged with the language model.
from google.genai import types

# TavilySearchResults: The LangChain tool for performing web searches using the Tavily API.
from langchain_community.tools import TavilySearchResults

# Markdown, display: Used for rendering rich text output in Jupyter notebooks.
from IPython.display import Markdown, display

In [7]:
# Tavily Search API: Set Up
# API Key Management: Always use environment variables or a secure vault for credentials!
# This line attempts to retrieve your Tavily API key from an environment variable.
# Replace 'YOUR_TAVILY_API_KEY' with your actual key if you're not using environment variables,
# but it's strongly recommended to set it securely (e.g., in a .env file or system-wide).
# You can obtain a free API key from https://www.tavily.com
TAVILY_API_KEY = os.environ.get("TAVILY_API_KEY")

if not TAVILY_API_KEY:
    raise ValueError(
        "TAVILY_API_KEY environment variable not set. "
        "Never hardcode API keys in your code. "
        "Set it securely in your environment or use a secrets manager."
    )

In [8]:
# Tavily LangChain tool: Initialization
# This tool allows the agent to perform live web searches using Tavily.
# We configure it for 'advanced' search depth and to include various data types.
tavily_tool_instance = TavilySearchResults(
    max_results=5,                # Limit the number of search results to 5
    search_depth="advanced",      # Use a more comprehensive search
    include_answer=True,          # Include a summarized answer from Tavily
    include_raw_content=True,     # Include the raw content of the search results
    include_images=True,          # Include images from search results (though not directly used by agent instructions here)
)

# Wrap it for ADK compatibility
# ADK agents can directly use tools wrapped by LangchainTool.
adk_tavily_tool = LangchainTool(tool=tavily_tool_instance)

In [9]:
# Latest Events Agent: Definition
# The instruction set is the core of the agent, guiding its behavior.
# It should be specific, structured, and ensure safe operation.
latest_events_agent = Agent(
    name="latest_events_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="An agent that uses the Tavily search tool to find and summarize the latest events, festivals, and activities for a given destination, providing up-to-date and user-friendly travel information.",
    instruction="""
        You are a highly helpful and detailed travel assistant specialized in finding events.
        Your primary goal is to provide accurate, comprehensive, and well-organized information about events.

        When a user asks about current or upcoming events, festivals, or activities at a destination, follow these steps strictly:
        1.  **Identify Key Information:** Extract the desired event type (e.g., food festivals, music concerts, art exhibitions, sports events, cultural celebrations), destination, and timeframe (e.g., "next month", "August 2025", "next week", "soon").
            *   **Timeframe Resolution:** If the timeframe is relative (e.g., "next month", "next week"), resolve it to specific calendar dates or a specific month/year *before* formulating the search query (e.g., if today is June 15, 2024, "next month" becomes "July 2024").
        2.  **Use Tavily Search:** Utilize the `tavily_search_results` tool to find the most recent and relevant information. Formulate precise search queries for Tavily using the resolved timeframe. For example, "food festivals London July 2024", "music concerts Berlin August 2025".
        3.  **Prioritize Official Sources:** Always aim to extract information from official event websites, reputable ticketing platforms, or well-known event listing sites.
        4.  **Filter Results by Location (CRUCIAL):** After performing the search, rigorously filter the results to ensure that all events listed are *strictly* located within the specified destination. **Do not include events from nearby cities, states, or countries if the user asked for a specific city.** For example, if the user asks for "San Francisco", do not include events from Oakland or Toronto.
        5.  **Summarize and Structure - IMPORTANT: FOLLOW THIS CLEANER FORMAT PRECISELY:**
            *   Start with a clear, friendly introduction.
            *   Present each event as a main item in a **Markdown unordered list (using asterisks `*`)**.\
            *   Each event entry MUST follow this exact Markdown structure for clarity and spacing:
                ```
                *   **[Event Name]**
                    *   Description: [Brief Description].
                    *   Dates: [Start Date] - [End Date or Single Date].
                    *   Location: [Venue/Area], [City].
                    *   [More Info](URL) (Optional: only include if a verified, direct URL for the event or its official listing is found)
                ```
            *   Ensure only the "Event Name" is bolded as a main heading for each event.
            *   If dates are a range, use "Start Date - End Date". If single day, use "Single Date".
            *   Organize the events **chronologically by start date** if multiple events are found.

            **Example Event Formatting (pay close attention to line breaks and reduced bolding):**
            ```
            *   **Taste of London**
                *   Description: A renowned food festival featuring top restaurants and culinary experiences.
                *   Dates: June 18 - June 22, 2025.
                *   Location: Regent's Park, London.
                *   [More Info](https://www.tasteoflondon.co.uk/)

            *   **Berlin Lollapalooza**
                *   Description: A major music festival with diverse artists and stages.
                *   Dates: August 9 - August 10, 2025.
                *   Location: Olympiapark and Olympiastadion, Berlin.
                *   [More Info](https://www.lollapaloozade.com/)
            ```
        6.  **Handle Missing Information:**
            *   If the timeframe is vague (e.g., "soon", "what's happening?"), default to searching for events in the *upcoming 1-2 months* from the current date. **Crucially, only include events that fall within this specified timeframe.** If you find too many, suggest a more specific timeframe.
            *   If no specific event type is requested, search for a broad range of popular events (e.g., "major events [city] [month]").
        7.  **No Results Found:** If, after searching, no relevant events are found after filtering by location and date, politely state that no events matching the criteria could be found and offer to search for different dates or event types.
        8.  **Maintain Tone:** Be concise, friendly, helpful, and EXACT.
        9.  **Safety and Redirection:** If the user asks about general travel advice or information not related to current events, politely redirect them to other resources. Never expose API keys or sensitive data in your responses.
""",
    tools=[adk_tavily_tool], # Assign the Tavily tool to this agent.
)

print("`latest_events_agent` successfully defined with comprehensive instructions.")

`latest_events_agent` successfully defined with comprehensive instructions.


In [10]:
# Modular Function for Agent Interaction
# This asynchronous function encapsulates the common logic for running an ADK agent interaction,
# making the main execution flow cleaner and more reusable.
async def run_adk_agent_interaction(
    agent: Agent,
    user_id: str,
    session_id: str,
    input_text: str,
    app_name: str = "default_app",
    session_service: InMemorySessionService = None,
) -> str:
    """
    Runs a single interaction with an ADK agent and returns its final text response.

    Args:
        agent: The ADK Agent instance to interact with.
        user_id: A unique identifier for the user initiating the interaction.
        session_id: A unique identifier for the conversation session.
                    Using a new ID for each distinct example ensures isolation.
        input_text: The textual message from the user.
        app_name: The name of the application using the agent. Defaults to "default_app".
        session_service: An optional InMemorySessionService instance. If None, a new
                         instance is created, making each call self-contained.

    Returns:
        The final text response from the agent as a string. Returns
        "No final response from agent." if no final response event is found.
    """
    # Use the provided session service if available, otherwise create a new one.
    # This flexibility allows for both isolated examples and continuous conversations.
    if session_service is None:
        session_service = InMemorySessionService()

    # Create a new session. In a multi-turn conversation, you would reuse
    # the same session_id across turns to maintain context.
    # For these distinct examples, we'll use unique session_ids.
    await session_service.create_session(app_name=app_name, user_id=user_id, session_id=session_id)

    # Format the user's input into a Content object, which is how messages
    # are structured for the agent.
    content = types.Content(role="user", parts=[types.Part(text=input_text)])

    # Initialize the Runner: This is the engine that connects the user's message
    # to the agent and manages the session state.
    runner = Runner(agent=agent, app_name=app_name, session_service=session_service)

    # Run the agent: This sends the user's message to the agent for processing.
    # The 'runner.run()' method returns an iterable of events that describe
    # the agent's actions and responses.
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    # Process the events to find and extract the final response from the agent.
    for event in events:
        if event.is_final_response():
            # Concatenate all parts in the final response
            response = ''.join(part.text for part in event.content.parts if part.text)
            return response
    return "No final response from agent."

print("Modular function `run_adk_agent_interaction` successfully defined.")

Modular function `run_adk_agent_interaction` successfully defined.


In [11]:
# Define common parameters for our agent interactions.
APP_NAME = "wanderwise_app"
USER_ID = "user_001"

# Create a common session service instance to reuse across multiple interactions.
# This helps if you want to test conversational continuity, though for distinct
# test cases, a new session ID is generated each time.
common_session_service = InMemorySessionService()

# Define a dictionary of user inputs for various test cases.
# Each entry includes a descriptive name and the user's query.
EVENT_TEST_CASES = {
    "Normal Case: Food festivals in Toronto next month":
        "What food festivals are happening in Toronto next month?",

    "Specific Case: Major art exhibitions in Milan this summer":
        "List major art exhibitions in Milan this summer.",

    "Future Date Request: Tech conferences in San Francisco in 2025":
        "Are there any tech conferences in San Francisco in 2025?",

    "Vague Timeframe: General events in Seoul soon":
        "What's happening in Seoul soon?",

    "Past Date Request: Festivals in Rio, January 2024":
        "What festivals happened in Rio in January 2024?",

    "No Results Expected: UFO Conventions, Hypothetical Location, Next Week":
        "Are there any major UFO conventions in Garetwasterland next week?",

    "Specific Date Query: Music events in New York City on July 4th, 2024":
        "Are there any music events in New York City on July 4th, 2024?",

    "Broad Event Type Query: Cultural events in Rome next year":
        "What cultural events are planned for Rome next year?",

    "Query for a less-known location/niche event":
        "Any major chess tournaments in Reykjavik next month?",

    "Unrelated Input: What is life?":
        "What is life?",

    "Invalid/Malicious Input: SQL Injection attempt":
        "DROP TABLE users; --",
}

print("------ [starting agent interactions] ------\n")

# Loop through each test case and run the agent.
for case_name, user_input in EVENT_TEST_CASES.items():
    # Create a unique session ID for each interaction to ensure isolated tests.
    session_id = f"{APP_NAME}_session_{case_name.replace(' ', '_').replace(',', '').replace(':', '').lower()}"

    print(f"\n--- Testing >> {case_name} ---")
    display(Markdown(f"**User Input:** '{user_input}'"))

    # Invoke the latest_events_agent using the modular helper function.
    agent_response = await run_adk_agent_interaction(
        agent=latest_events_agent,
        user_id=USER_ID,
        session_id=session_id,
        input_text=user_input,
        app_name=APP_NAME,
        session_service=common_session_service # Pass the common session service
    )

    # Display the agent's response.
    display(Markdown(f"**Agent Response:**\n\n{agent_response}"))
    print(f"\n--- Testing Complete >> {case_name} ---")


print("\n------ [all agent interactions complete] ------")

------ [starting agent interactions] ------


--- Testing >> Normal Case: Food festivals in Toronto next month ---


**User Input:** 'What food festivals are happening in Toronto next month?'

**Agent Response:**

Here are some food festivals happening in Toronto in July 2024:

*   **International Food + Drink Festival**
    *   Description: A food festival with a variety of international cuisines.
    *   Dates: July 12 - July 14, 2024.
    *   Location: Mel Lastman Square, Toronto.
    *   [More Info](https://curiocity.com/pop-up-food-festivals-toronto-2024/)

*   **Festival of India**
    *   Description: A celebration of Indian culture with vegetarian cuisine.
    *   Dates: July 13 - July 14, 2024.
    *   Location: Centre Island, Toronto.
    *   [More Info](https://theexploringfamily.com/locals-lounge/delicious-food-festivals-near-toronto/)

*   **Foodville Afro-Caribbean Food Festival**
    *   Description: A festival celebrating Afro-Caribbean cuisine.
    *   Dates: July 20, 2024.
    *   Location: Stackt Market, Toronto.



--- Testing Complete >> Normal Case: Food festivals in Toronto next month ---

--- Testing >> Specific Case: Major art exhibitions in Milan this summer ---


**User Input:** 'List major art exhibitions in Milan this summer.'

...

**Agent Response:**

I am designed to provide information about current events, festivals, and activities. For information about the meaning of life, you may want to consult a philosophical or spiritual resource.



--- Testing Complete >> Unrelated Input: What is life? ---

--- Testing >> Invalid/Malicious Input: SQL Injection attempt ---


**User Input:** 'DROP TABLE users; --'

**Agent Response:**

That is not a request I can handle. I am designed to find information about events, festivals, and activities. I cannot execute SQL commands or manage databases.



--- Testing Complete >> Invalid/Malicious Input: SQL Injection attempt ---

------ [all agent interactions complete] ------


In [ ]:
# NOTE: Some incorrect or incomplete results observed in this notebook may be attributed to limitations of the **Tavily** system. 
# Please consider verifying critical outputs through alternative sources or methods if accuracy is essential.

In [12]:
# Congratulations 🎉 
# You've just built and run an agent that uses the Tavily third-party search tool via LangChain and ADK to fetch live event data for your itinerary.
# This demonstrates the power of combining LLM reasoning with external tools to create
# truly intelligent and practical applications.
# Try modifying the prompt or agent instructions, and see how the results change!